In [1]:
!pip3 install sympy networkx matplotlib pprint numpy scipy

  Using cached matplotlib-2.1.2-cp35-cp35m-manylinux1_x86_64.whl
  Using cached numpy-1.14.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached scipy-1.0.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached pytz-2017.3-py2.py3-none-any.whl
  Using cached cycler-0.10.0-py2.py3-none-any.whl
  Using cached pyparsing-2.2.0-py2.py3-none-any.whl


In [2]:
import sympy as sm
import networkx as nx
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
import math
from copy import deepcopy
import itertools

In [3]:
round_count = 2
cer = 10
eps = 1e-3
lam = 10
steps = 35

files = [
    'gpa-1.txt',
    'gpa-2.txt',
    'undetermine-system.txt'
]

values_gpa = {
    '1' : [
        ['p2', 10],
        ['p4', 10],
        ['q1', 4],
        ['q2', 4],
        ['q3', 4],
        ['q4', 4],
        ['q5', 4]
    ],
    '2': [
        ['p2', 160],
        ['q1', 60],
        ['q2', 50],
        ['q3', 50]
    ],
    '3': [
        ['p2', 160],
        ['p3', 100],
        ['q1', 60],
        ['q2', 50],
        ['q3', 50],
        ['q4', 10]
    ],
    '4':[
        ['p2', 160],
        ['p3', 100],
        ['p4', 100],
        ['q1', 60],
    ]
}

In [4]:
def cacl_jac(all_symbols=[], equations=[]):
    jacobi = sm.zeros(len(all_symbols), len(all_symbols))
    for var_i in range(len(all_symbols)):
        var = all_symbols[var_i]
        for eq_i in range(len(equations)):
            eq = equations[eq_i]
            jacobi[eq_i, var_i] = sm.diff(eq, var)
        
    jacobian = jacobi.det()
    
    return jacobi, jacobian

In [5]:
def disp_graph_with_custom_labels(gr=None, 
                                  nodes_labels={}, 
                                  edges_labels={},
                                  fig_size=8,
                                  node_label=None, 
                                  edge_label=None):
    plt.figure(figsize=(fig_size, fig_size))
    
    if not edge_label == None and not edge_label == '':
        edges_labels = nx.get_edge_attributes(G=gr, name=edge_label)
        
    if not node_label == None and not node_label == '':
        nodes_labels = nx.get_node_attributes(G=gr, name=node_label)

    layout = nx.shell_layout(gr)
    
    nx.draw(G=gr, pos=layout, node_size=1500)
    nx.draw_networkx_edge_labels(G=gr, 
                                 pos=layout, 
                                 edge_labels=edges_labels, 
                                 font_family='sans-serif')
    nx.draw_networkx_labels(G=gr, 
                            pos=layout, 
                            labels=nodes_labels, 
                            font_family='sans-serif')
    plt.show()

In [6]:
def subs_to_graph_new_values(gr=None, values=[]):
    q = nx.get_edge_attributes(G=gr, name='q')
    p = nx.get_node_attributes(G=gr, name='p')

    for sub_expr in values:
        sym = sub_expr[0]
        val = round(sub_expr[1], round_count)

        for edge_param in list(q.items()):
            if sym == str(edge_param[1]):
                q[edge_param[0]] = val

        for node_param in list(p.items()):
            if sym == str(node_param[1]):
                p[node_param[0]] = val

#     print(q)
#     print(p)

    nx.set_edge_attributes(G=gr, name='q', values=q)
    nx.set_node_attributes(G=gr, name='p', values=p)

In [7]:
def disp_graph(gr=None, edge_label='q',
               node_label='p',
               fig_size=8):
    
    plt.figure(figsize=(fig_size, fig_size))
    
    edges_labels = nx.get_edge_attributes(G=gr, 
                                          name=edge_label)
    nodes_labels = nx.get_node_attributes(G=gr, 
                                          name=node_label)
    
    layout = nx.shell_layout(gr)
    
    nx.draw(G=gr, pos=layout, 
            node_size=1500)
    nx.draw_networkx_edge_labels(G=gr, 
                                 pos=layout, 
                                 edge_labels=edges_labels, 
                                 font_family='sans-serif')
    nx.draw_networkx_labels(G=gr, 
                            pos=layout, 
                            labels=nodes_labels, 
                            font_family='sans-serif')
    plt.show()

In [8]:
def read_graph(path=''):
    all_symbols = []

    start_nodes = set()
    terminal_nodes = set()
    all_nodes = set()
    
    gpa = nx.DiGraph()

    with open(path, 'r') as input:
        line = input.readline()

        params = list(map(lambda x : int(x), 
                          line.split(' ')))
        V, E = params[0], params[1]

        graph_params = {
            'nodes' : dict(),
            'edges' : dict()
        }

        for i in range(V):
            params = list(map(lambda x : int(x), 
                              input.readline().split(' ')))
            node = int(params[0])
    
            start_nodes.add(node)
            terminal_nodes.add(node)
            all_nodes.add(node)
            
            if len(params) == 1:
                p_i = sm.symbols('p' + str(i + 1))
                known = False
                all_symbols.append(p_i)
            else:
                p_i = params[1]
                known = True
            gpa.add_node(node, 
                         p=p_i,
                         q=0,
                         known_p=known,
                         known_q=True,
                         influx=0,
                         influx_consumption=False,
                         final_pressure=False,
                         code=str(i + 1),
                         var=sm.symbols('p' + str(i + 1)))

        for i in range(E):
            params = list(map(lambda x : int(x), 
                              input.readline().split(' ')))

            u = int(params[0])
            v = int(params[1])
            l = params[2]
            d = params[3]
            
            if v in start_nodes:
                start_nodes.remove(v)
            
            if u in terminal_nodes:
                terminal_nodes.remove(u)

            if len(params) == 4:
                q_i = sm.symbols('q' + str(i + 1))
                known = False
                all_symbols.append(q_i)
            else:
                q_i = params[4]
                known = True
            
            if not gpa.nodes[u]['q'] == 0:
                gpa.nodes[u]['q'] -= q_i
                gpa.nodes[u]['known_q'] = (gpa.nodes[u]['known_q'] and 
                                           not isinstance(q_i, sm.Symbol))
            else:
                gpa.nodes[u]['q'] = q_i
                gpa.nodes[u]['known_q'] = (gpa.nodes[u]['known_q'] and 
                                           not isinstance(q_i, sm.Symbol))
            
            if not gpa.nodes[v]['q'] == 0:
                gpa.nodes[v]['q'] -= q_i
                gpa.nodes[v]['known_q'] = (gpa.nodes[v]['known_q'] and 
                                           not isinstance(q_i, sm.Symbol))
            else:
                gpa.nodes[v]['q'] = q_i
                gpa.nodes[v]['known_q'] = (gpa.nodes[v]['known_q'] and 
                                           not isinstance(q_i, sm.Symbol))
            
            gpa.add_edge(u_of_edge=u, 
                         v_of_edge=v,
                         q=q_i, 
                         L=l, 
                         D=d, 
                         known=known, 
                         code=str(i + 1), 
                         var=sm.symbols('q' + str(i + 1)))
            
            
        for node_index in gpa.nodes():
            node = gpa.nodes[node_index]
            
            node['final_pressure'] = not isinstance(node['p'], sm.Symbol)
            node['influx_consumption'] = isinstance(node['p'], sm.Symbol)
            
            if node_index in terminal_nodes or node_index in start_nodes:
                node['influx'] = node['q']
            
    return gpa, all_symbols

In [9]:
def get_equations(gr=None, all_symbols=[]):
    q = nx.get_edge_attributes(G=gr, name='q')
    p = nx.get_node_attributes(G=gr, name='p')
    l = nx.get_edge_attributes(G=gr, name='L')
    d = nx.get_edge_attributes(G=gr, name='D')
    
    equations = []

    for edge, param in q.items():
        qc = param
        pcs = p[edge[0]]
        pcf = p[edge[1]]
        dc = d[edge]
        lc = l[edge]

        counter = 0

        eq = pcs * pcs - pcf * pcf - qc * qc * lc * dc ** 5

        fl = True
        for sym in all_symbols:
            expr = sm.diff(eq, sym)
            fl = fl and expr == 0

        if not fl:
            equations.append(eq)

    edge_list = list(gr.edges())
    temp = np.zeros((len(edge_list), 2))
    for i in range(len(edge_list)):
        edge = edge_list[i]
        temp[i][0] = edge[0]
        temp[i][1] = edge[1]

    edges = temp

    for node in gr.nodes():

        eq = 0
        counter = 0
        for temp_edge in edges:
            if temp_edge[0] == node:
                qc = q[(temp_edge[0], temp_edge[1])]
                eq += qc
                counter += 1
            if temp_edge[1] == node:
                qc = q[(temp_edge[0], temp_edge[1])]
                eq -= qc
                counter += 1

        fl = True
        for sym in all_symbols:
            expr = sm.diff(eq, sym)
            fl = fl and expr == 0

        if not fl and counter > 1:
            equations.append(eq)

    for i in range(len(equations)):
        print('equation ' + 
              str(i + 1) + 
              ' : ' + 
              str(equations[i]))

    return equations

In [10]:
def get_equations_from_node(gr=None, all_symbols=[]):
    q = nx.get_edge_attributes(G=gr, name='q')
    p = nx.get_node_attributes(G=gr, name='p')
    l = nx.get_edge_attributes(G=gr, name='L')
    d = nx.get_edge_attributes(G=gr, name='D')
    
    equations = []

    for edge, param in q.items():
        qc = param
        pcs = p[edge[0]]
        pcf = p[edge[1]]
        dc = d[edge]
        lc = l[edge]

        counter = 0

        eq = pcs * pcs - pcf * pcf - qc * qc * lc * dc ** 5

        fl = True
        for sym in all_symbols:
            expr = sm.diff(eq, sym)
            fl = fl and expr == 0

        if not fl:
            equations.append(eq)
            
    for node_index in gr.nodes():
        node = gr.nodes[node_index]
        if node['influx'] == 0:
            equations.append(node['q'])
    
    return equations

In [11]:
def get_newton_equations(equations=[], all_symbols=[]):
    jacobi, jacobian = cacl_jac(all_symbols=all_symbols, 
                                equations=equations)
    
    eqs = []
    eq = sm.Matrix(deepcopy(equations))
    
    for i in range(len(all_symbols)):
        f_i = deepcopy(jacobi)
        f_i[:, i] = eq
        
        f_i = f_i.det()
        f_i = f_i / jacobian

        
        eqs.append(sm.simplify(f_i))
        
    return eqs

In [12]:
def subs(values=[], eqs=[]):
    ans = [float(eq.subs(values)) 
           for eq in eqs]
    
    return ans

In [13]:
def partial_subs(values=[], eqs=[]):  
    ans = [eq.subs(values) for eq in eqs]
    
    return ans

In [14]:
def solving(values=[], eqs=[], steps=25, eps=0.001):
    cer = 1
    ers = []

    while cer > eps and steps > 0:
        er = []
        
        try:
            
            nv = []

            ans = subs(eqs=eqs, values=values)

            for i in range(len(values)):
                val = values[i][1] - ans[i]
                sym = values[i][0]

                nv.append((sym, val))
                er.append(math.fabs(ans[i]))

            values = nv    

            er = np.linalg.norm(np.array(er))
            ers.append(er)
            cer = er

            steps -= 1

        except Exception as ex:
            break
    
    return values, ers

# Реализация метода Ньютона

In [15]:
def calc(path='', 
         values=[], 
         pre_disp_graph=False, 
         post_disp_graph=False, 
         disp_error_plot=False, 
         disp_equations_info=False):
    
    gpa, all_symbols = read_graph(path=path)
    
    if pre_disp_graph:
        disp_graph_with_custom_labels(gr=gpa, 
                                      edge_label='q', 
                                      node_label='p')
    
    equations = get_equations_from_node(gr=gpa, 
                              all_symbols=all_symbols)
    
    if disp_equations_info:
        print()
        print('Переменных :', len(all_symbols), all_symbols)
        print('Уравнений :', len(equations))
        pprint(equations)
    
    
    eqs = get_newton_equations(all_symbols=all_symbols, 
                               equations=equations)
    values, ers = solving(values=values, eqs=eqs)
    subs_to_graph_new_values(values=values, gr=gpa)
    
    if post_disp_graph:
        disp_graph(gr=gpa, edge_label='q', node_label='p')
    
    if disp_error_plot:
        plt.figure(figsize=(10, 10))
        plt.plot(range(len(ers)), ers, 'b')
        plt.plot(range(len(ers)), ers, 'ro')
        plt.ylabel(s='Норма изменения')
        plt.xlabel(s='Номер шага')
        plt.grid()
        plt.show()
        print('\nError : ' + str(min(ers)))
    
    return gpa

In [16]:
gpa = calc(path='gpa-4.txt', 
           values=values_gpa['4'])

In [ ]:
disp_graph_with_custom_labels(gr=gpa, 
                              edge_label='q', 
                              node_label='q')

In [18]:
def get_AF_AL(A=None):
    AF = sm.zeros(A.shape[0], A.shape[1])
    AL = sm.zeros(A.shape[0], A.shape[1])

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            val = int(A[i, j])
            if val == 1:
                AF[i, j] = 1
            elif val == -1:
                AL[i, j] = 1

    AF_T = sm.Matrix(AF.T)
    AL_T = sm.Matrix(AL.T)
    
    return (AF, AL)

In [19]:
def phi(ps, pf, D, L):
    return (ps ** 2 - pf ** 2)

In [20]:
def get_edge_index(gr=None, edge=()):
    return list(gr.edges()).index(edge)

In [21]:
def get_edge_by_index(gr=None, index=0):
    return list(gr.edges())[index]

In [22]:
def get_node_by_index(gr=None, index=0):
    return gr.nodes[index]

In [23]:
def replace_cols(A=None, i=0, j=0):
    shape = A.shape
    
    A_copy = deepcopy(A)
    
    rows = shape[0]
    cols = shape[1]
    
    if i < cols and j < cols:
        A_copy[:, i] = A[:, j]
        A_copy[:, j] = A[:, i]
        
        return A_copy

In [24]:
def replace_rows(A=None, i=0, j=0):
    shape = A.shape
    
    A_copy = deepcopy(A)
    
    rows = shape[0]
    cols = shape[1]
    
    print(shape, i, j)
    
    if i < rows and j < rows:
        A_copy[j, :] = A[i, :]
        A_copy[i, :] = A[j, :]
        
        return A_copy

In [25]:
def set_cols_order(A=None, cols_order=[]):
    T = deepcopy(A)
    for pos, index in zip(cols_order, 
                          range(len(cols_order))):
        T[:, index] = A[:, pos]
        
    return T

In [26]:
def set_rows_order(A=None, rows_order=[]):
    T = deepcopy(A)
    for pos, index in zip(rows_order, 
                          range(len(rows_order))):
        T[index, :] = A[pos, :]
        
    return T

In [27]:
def get_all_matrix(gpa=None, phi=None):
    
    A = nx.incidence_matrix(G=gpa, 
                            oriented=True).todense()
    
    P = [ gpa.nodes[node]['var'] 
         for node in gpa.nodes() ]
    P = sm.Matrix(P)
    
    known_nodes_q = [ node - 1 for node in gpa.nodes() 
                   if gpa.nodes[node]['influx_consumption']]
    
    unknown_nodes_q = [ node - 1 for node in gpa.nodes() 
                     if not gpa.nodes[node]['influx_consumption']]
    known_nodes_p = [ node - 1 for node in gpa.nodes() 
                   if gpa.nodes[node]['final_pressure']]
    unknown_nodes_p = [ node - 1 for node in gpa.nodes() 
                     if not gpa.nodes[node]['final_pressure']]
    knowledges_nodes = {
        'p':{
            'known' : known_nodes_p,
            'unknown' : unknown_nodes_p
        },
        'q':{
            'known' : known_nodes_q,
            'unknown' : unknown_nodes_q
        }
    }
    
    eq = []
    
    A_s = get_AF_AL(A=A)
    
    AF_T = sm.Matrix(A_s[0].T)
    AL_T = sm.Matrix(A_s[1].T)
    
    PF = AF_T * P
    PL = AL_T * P

    for pf, ps in zip(PF, PL):
        eq.append(phi(ps, pf, 1, 1))
        
    X = sm.Matrix(eq)
    
    d_F_i = []
    d_L_i = []

    diff_vars = []
    for edge_index, eq in zip(gpa.edges(), X):
        node_s = gpa.node[edge_index[0]]['var']
        node_f = gpa.node[edge_index[1]]['var']
        
        eq_s = sm.diff(eq, node_s)
        eq_f = -sm.diff(eq, node_f)
        
        d_F_i.append(eq_s)
        d_L_i.append(eq_f)
    
    d_F_i = sm.diag(*d_F_i)
    d_L_i = sm.diag(*d_L_i)
    
    return (A, X,
            (AF_T, AL_T), 
            (d_F_i, d_L_i), 
            P, 
            knowledges_nodes)

In [28]:
def get_Q(gr=None):    
    Q = []
    for node in gr.nodes():
        Q.append(gr.nodes[node]['influx'])
    
    return sm.Matrix(Q)

In [29]:
def get_P(gr=None):    
    P = []
    for node in gr.nodes():
        P.append(gr.nodes[node]['p'])
    
    return sm.Matrix(P)

In [30]:
gpa, syms = read_graph(path='gpa-1.txt')
res = get_all_matrix(gpa=gpa, phi=phi)

In [31]:
A = -1 * res[0]
X = res[1]
AF_T = res[2][0]
AL_T = res[2][1]
d_F = res[3][0]
d_L = res[3][1]
P = res[4]

In [32]:
nodes_p = res[5]['p']['known'] + res[5]['p']['unknown']
nodes_q = res[5]['q']['known'] + res[5]['q']['unknown']

In [33]:
P = get_P(gr=gpa)
Q = get_Q(gr=gpa)

P = set_rows_order(A=P, 
                   rows_order=res[5]['p']['unknown'] + res[5]['p']['known'])
Q = set_rows_order(A=Q, 
                   rows_order=res[5]['q']['known'] + res[5]['q']['unknown'])

In [34]:
P

Matrix([
[p2],
[p4],
[12],
[ 4],
[ 7],
[ 6]])

In [35]:
Q

Matrix([
[ 0],
[ 0],
[q1],
[q2],
[q4],
[q5]])